In [11]:
from utils import auto_wonder_pipeline, process_data_rd, filter

observed_outcome = [
    "Diseases of the circulatory system",
    "Diseases of the respiratory system",
]
short_outcome = ["circulatory system", "respiratory system"]

data = auto_wonder_pipeline(
    observed_outcome=observed_outcome,
    short_outcome=short_outcome,
)

data_65 = auto_wonder_pipeline(
    observed_outcome=observed_outcome,
    short_outcome=short_outcome,
    identifer="65+",
    columns = ["State", "Month Code", "ICD Chapter", "Deaths"]
)

data

cirdeaths       date  resdeaths  totaldeaths
state time                                                    
AK    2010-01-01       78.0 2010-01-01       25.0        103.0
      2010-02-01       74.0 2010-02-01       23.0         97.0
      2010-03-01       64.0 2010-03-01       23.0         87.0
      2010-04-01       75.0 2010-04-01       41.0        116.0
      2010-05-01       62.0 2010-05-01       29.0         91.0
...                     ...        ...        ...          ...
WY    2020-08-01      118.0 2020-08-01       47.0        165.0
      2020-09-01      145.0 2020-09-01       43.0        188.0
      2020-10-01      144.0 2020-10-01       56.0        200.0
      2020-11-01      122.0 2020-11-01       41.0        163.0
      2020-12-01      143.0 2020-12-01       61.0        204.0

[6732 rows x 4 columns]

In [12]:
import pandas as pd
data_ = pd.read_excel('data/ml.xlsx', sheet_name=None)

key_mapper = {
    'FEMA - Major Disaster - Fire': 'fire',
    'FEMA - Major Disaster - Earthqu': 'earthquake',
    'FEMA - Major Disaster - Hurrica': 'hurricane',
    'FEMA - Major Disaster - Tornado': 'tornado',
    'FEMA - Major Disaster - Flood': 'flood',
    }
data_ = {key_mapper.get(old_key, old_key): value for old_key, value in data_.items()}

df_disaster = data_['fire'].loc[:,['State', 'Declaration Date', 'Incident Type']]

from countries_code import code
df_disaster['State'] = df_disaster['State'].map(code)
df_disaster = df_disaster.dropna()
df_disaster.head()

,State,Declaration Date,Incident Type
0,CA,2021-09-12,Fire
1,CA,2021-08-24,Fire
2,WA,2021-02-04,Fire
3,CO,2021-01-15,Fire
4,MT,2021-09-30,Fire


We don't have data after 2020, so no study is possible

In [45]:
df_disaster = df_disaster[df_disaster["Declaration Date"] < "2020"]

In [46]:
df_temporel = {
    (date, state): process_data_rd(data, state=state, disaster_date=date)
    for date, state in zip(df_disaster["Declaration Date"], df_disaster["State"])
}

df_temporel_65 = {
    (date, state): process_data_rd(data_65, state=state, disaster_date=date)
    for date, state in zip(df_disaster["Declaration Date"], df_disaster["State"])
}


[filter(x) for x in df_temporel.values()]
[filter(x) for x in df_temporel_65.values()]
pass

In [98]:
from typing import Any, List
import pandas as pd
def plot(fig:Any, df: pd.DataFrame, row:int, col:int, disaster_time:str, state: str, category:str, other_events:List[int]):
    # Left y
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df.cirdeaths,
            name="Diseases of the circulatory system",
            mode="markers",
            marker=dict(
                color="rgb(27,158,119)",
            ),
            marker_symbol=symbols[namestems.index("circle-cross")],
            marker_line_width=2,
            marker_size=10,
            # showlegend=False,
        ),
        secondary_y=False,
        row=row,
        col=col,
    )

    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df.cirFiltered,
            marker=dict(
                color="rgb(27,158,119)",
            ),
        ),
        secondary_y=False,
        row=row,
        col=col,
    )
    #Right y
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df.resdeaths,
            name="Diseases of the respiratory system",
            mode="markers",
            marker_symbol=symbols[namestems.index("circle-x")],
            marker=dict(
                color="rgb(297,95,2)",
            ),
            marker_line_width=1.5,
            marker_size=10,
            # showlegend=False,
        ),
        secondary_y=True,
        row=row,
        col=col,
    )

    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df.resFiltered,
            marker=dict(
                color="rgb(297,95,2)",
            ),
        ),
        secondary_y=True,
        row=row,
        col=col,
    )

    # Set x-axis title
    fig.update_xaxes(title_text=f"{disaster_time} in {state} on {category}", range=[-365.0, 365], row=row, col=col)

    # Set y-axes titles
    if col%2:
        fig.update_yaxes(
        title_text='<b style="color:rgb(297,95,2)">Respiratory Axis x</b>',
        secondary_y=True, row=row, col=col,
    )
    else:
        fig.update_yaxes(
            title_text='<b style="color:rgb(27,158,119)">Circulatory axis &#10010;</b> ',
            secondary_y=False, row=row, col=col,
        )
    
    fig.add_vline(x=0.0, line_color="rgb(117,112,79)", line_dash="dash")
    [fig.add_vline(x=shift, line_color="rgb(117,112,79)", line_dash="dash") for shift in other_events]

In [102]:
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator
import plotly.graph_objects as go
from typing import Dict

raw_symbols = SymbolValidator().values
namestems = []
namevariants = []
symbols = []
for i in range(0, len(raw_symbols), 3):
    name = raw_symbols[i + 2]
    symbols.append(raw_symbols[i])
    namestems.append(name.replace("-open", "").replace("-dot", ""))
    namevariants.append(name[len(namestems[-1]) :])

# def plot(df_temporel: Dict[pd.Timestamp: pd.DataFrame]):
"""Plot at each row death count in each category.

Args:
df_temporel (_type_): _description_
"""
rows = len(df_temporel)
cols = 2


# Create figure with secondary y-axis
fig = make_subplots(
    rows=rows,
    cols=2,
    specs=[[{"secondary_y": True}, {"secondary_y": True}] for _ in range(rows)],
)

liste_ = list(df_temporel.keys())[:rows]
from tqdm import tqdm
for row, (date, state) in enumerate(tqdm(liste_), start=1):
    shifts=[]
    max_shift = 365
    k = row-1
    
    time = str(date).split(" ")[0]
    for i in list(range(k-1,-1,-1))+list(range(k+1,len(liste_))):
        neighbor_time, neighbor_state = liste_[i]
        if neighbor_state==state:
            shifts.append(int((neighbor_time-pd.to_datetime(time))/ np.timedelta64(1, "D")))
        
    for col in [1, 2]:
        plot(
            fig=fig,
            df=df_temporel[(date, state)] if col % 2 else df_temporel_65[(date, state)],
            row=row,
            col=col,
            disaster_time=time,
            state=state,
            other_events=shifts,
            category= "all" if col % 2 else "65+"
        )


fig.update_layout(title_text="Fire disasters", height= 1000*rows/3,plot_bgcolor='white')
fig.update_traces(showlegend=False)

# fig.update(layout_xaxis_range = [-365,365])

fig.show()

100%|██████████| 14/14 [22:57<00:00, 98.40s/it] 


In [103]:
fig.write_html("ml.html")

In [104]:
fig.write_image("ml.png")

In [81]:
liste_ = list(df_temporel.keys())

k = 3
time = '2018-06-25'
state = 'CA'

shifts=[]

max_shift = 365

for i in list(range(k-1,-1,-1))+list(range(k+1,len(liste_))):
    neighbor_time, neighbor_state = liste_[i]
    if neighbor_state==state:
        shifts.append(int((neighbor_time-pd.to_datetime(time))/ np.timedelta64(1, "D")))
        

In [91]:
import numpy as np

int((neighbor_time-pd.to_datetime(time))/ np.timedelta64(1, "D"))

-2188

In [67]:
for i in range(0-1,-1,-1):
    print(i)

In [70]:
list(range(0,5,2))

[0, 2, 4]

In [50]:
list(df_temporel.keys())[:3]

[(Timestamp('2018-11-12 00:00:00'), 'CA'),
 (Timestamp('2018-08-04 00:00:00'), 'CA'),
 (Timestamp('2018-01-02 00:00:00'), 'CA')]

In [183]:
fig.layout

Layout({
    'shapes': [{'line': {'color': 'rgb(117,112,79)', 'dash': 'dash'},
                'type': 'line',
                'x0': 0.0,
                'x1': 0.0,
                'xref': 'x',
                'y0': 0,
                'y1': 1,
                'yref': 'y domain'},
               {'line': {'color': 'rgb(117,112,79)', 'dash': 'dash'},
                'type': 'line',
                'x0': 0.0,
                'x1': 0.0,
                'xref': 'x',
                'y0': 0,
                'y1': 1,
                'yref': 'y domain'},
               {'line': {'color': 'rgb(117,112,79)', 'dash': 'dash'},
                'type': 'line',
                'x0': 0.0,
                'x1': 0.0,
                'xref': 'x2',
                'y0': 0,
                'y1': 1,
                'yref': 'y3 domain'},
               {'line': {'color': 'rgb(117,112,79)', 'dash': 'dash'},
                'type': 'line',
                'x0': 0.0,
                'x1': 0.0,
                '

In [175]:
fig2 = make_subplots(
    rows=2, cols=1, specs = (
            [[{"secondary_y": True} for _ in range(1)] for _ in range(2)]
        )
)

fig2.add_trace(fig,
              row=1, col=1)

fig2.add_trace(fig, row=2, col=1)



fig2.show()

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [Figure({
    'data': [{'marker': {'color': 'rgb(27,158,119)', 'line': {'width': 2}, 'size': 10, 'symbol': 27},
              'mode': 'markers',
              'name': 'Diseases of the circulatory system',
              'showlegend': False,
              'type': 'scatter',
              'x': array([-3237, -3206, -3178, -3147, -3117, -3086, -3056, -3025, -2994, -2964,
                          -2933, -2903, -2872, -2841, -2813, -2782, -2752, -2721, -2691, -2660,
                          -2629, -2599, -2568, -2538, -2507, -2476, -2447, -2416, -2386, -2355,
                          -2325, -2294, -2263, -2233, -2202, -2172, -2141, -2110, -2082, -2051,
                          -2021, -1990, -1960, -1929, -1898, -1868, -1837, -1807, -1776, -1745,
                          -1717, -1686, -1656, -1625, -1595, -1564, -1533, -1503, -1472, -1442,
                          -1411, -1380, -135

In [2]:
from wonder_utils import DataPloter

from typing import Dict, List, Any
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import pandas as pd
import numpy as np
import os


class Death_Data(DataPloter):
    """
    Available features to select:
        state, month, age_category, ICD chapter

    Available features to plot:
        deaths, suicide_proportion, suicide_per_100k

    The data pipeline works as following:"""

    def __init__(
        self,
        data_folder: str = "Data",
        indexer_columns: List[str] = [
            "state",
            "date",
            "age_group",
            "icd",
        ],
        drop_cols: List[str] = [
            "Year Code",
            "State Code",
            "Month",  # keep month code
            "ICD Chapter Code",
            "UCD - ICD Chapter Code",
            "Crude Rate",
        ],
        reject_list: List[str] = ["test"],
    ) -> None:

        super(Death_Data, self).__init__(
            data_folder=data_folder,
            indexer_columns=indexer_columns,
            drop_cols=drop_cols,
            reject_list=reject_list,
        )

    def file_to_dataframe(
        self,
        data_folder: str,
        file: str,
        rename_mapper: Dict[str, str] = {
            "Month Code": "date",
            "ICD Chapter": "icd",
            "UCD - ICD Chapter": "icd",
            "Deaths": "deaths",
        },
    ) -> pd.DataFrame:
        """CDC Wonder txt file into dataframe
        Args:
            data_folder (str): where the data files are stored
            file (str): file that we want to process
            rename_mapper (_type_, optional): dictionnary to rename
                the columns.
                Defaults to {"Single Race 6": "race",
                             "Race": "race",
                             "Gender": "gender",
                             "Residence HHS Region Code": "hhs",
                             "HHS Region Code": "hhs",
                             "Population": "population",
                             "Year": "year", "Deaths": "deaths",
                             "Hispanic Origin": "ethnicity",
                             "Age Adjusted Rate": "age_adjusted_rate",}.

        Returns:
            pd.DataFrame: converted file into a pandas dataframe
        """
        process_line = lambda line: line.strip().replace('"', "").split("\t")

        lines = []
        with open(f"{data_folder}/{file}", "r") as f:
            for line in iter(lambda: f.readline().rstrip(), '"---"'):
                #CDC wonder add a total line despite we did not ask, 
                #ading one column sometime and breaking the pipeline
                if "Total" in line:
                    continue
                lines.append(process_line(line))
        # column is on the header, remove corner named notes
        res = pd.DataFrame(lines[1:], columns=lines[0][1:]).rename(
            columns=rename_mapper
        )
        # If Age Adjusted Rate is missing, fill with NaN
        if "age_adjusted_rate" not in res.columns:
            res["age_adjusted_rate"] = np.nan
        return res

    def processor(
        self,
        x: pd.DataFrame,
        force_numeric: List[str] = ["population", "Crude Rate"],
    ) -> pd.DataFrame:
        """Process dataframes: convert columns dtype, compute new features.
        Args:
            x (pd.DataFrame): dataframe that we want to process
            force_numeric (List[str], optional): force these columns
                into numerical columns.
                Defaults to ["population", "Crude Rate"].

        Returns:
            pd.DataFrame: processed dataframe

        """

        # Automatically convert numeric columns to float/int type
        if self.numeric_columns.empty:
            self.numeric_columns = x.apply(
                lambda s: pd.to_numeric(s.replace(np.nan, 0), errors="coerce")
                .notnull()
                .all()
            )

            # compare with the force numeric and prevent numeric columns
            self.convert_cols = [
                col
                for col, bool_ in self.numeric_columns.items()
                if (bool_ or col in force_numeric)  # and col != "year"
                # keep year as str to prevent unexpected ticks on plots
            ]

        x.columns = self.numeric_columns.index

        x.date = pd.to_datetime(x.date)#.str.extract("(\d+)")
        # x = x.replace(
        #     {
        #         "Not Hispanic or Latino": "Non-Hispanic",
        #         "Hispanic or Latino": "Hispanic",
        #         "Not Applicable": np.nan,
        #         "Unreliable": np.nan,
        #         "Asian or Pacific Islander": "API",
        #         "Asian": "API",
        #         "Black or African American": "Black",
        #         "Native Hawaiian or Other Pacific Islander": "API",
        #     }
        # )

        x[self.convert_cols] = x[self.convert_cols].apply(
            pd.to_numeric, errors="coerce"
        )
 
        x = x.loc[
            ~pd.concat(
                [x.eq(forbidden).any(axis=1) for forbidden in self.reject_list], axis=1
            ).any(axis=1)
        ]
        return x
        
    

    def load_data(
        self,
        drop_cols: List[str] = [],
        identifier: str = "death",
        data_folder: str = "data/CDC",
    ) -> Dict[str, pd.DataFrame]:
        """
        Automatically load all files containing identifier.
        This method is specific to the suicie_rate project and should be
        rewritten for other projects.

        Args:
            drop_cols (List[str], optional): Select what columns
                should be dropped. Defaults to [].
            identifier (str, optional): used to identify which files
                should be process. Defaults to "Data".

        Returns:
            Dict[str, pd.DataFrame]: dictionnary containing specific
                name of files and its associated dataframe.
        """
        available_files = os.listdir(data_folder)
        raw_data = [file for file in available_files if identifier in file]

        data = {
            (entry.split()[-1].split(".")[0], entry.split()[-2]): entry
            for entry in raw_data
        }

        age_strats = set(key[0] for key in data.keys())  # {'10-19', 'Overall', ...}

        dataframes = {
            age_strat: pd.concat(
                map(
                    self.processor,
                    [
                        self.file_to_dataframe(self.data_folder, file).assign(
                            age_strat=age_strat
                        )
                        for key_tuple, file in data.items()
                        if key_tuple[0] == age_strat
                    ],
                ),
                axis=0,
            )#.drop(columns=[x for x in drop_cols)
            for age_strat in age_strats
        }
        
        print("o")
        
        return {key: df.drop(df.filter(drop_cols), axis=1) for key, df in dataframes.items()}


test = Death_Data(data_folder="data/CDC")

o


In [11]:
disaster_date = "2015-09-01"
dd[dd.date=="disaster_date"]

,State,date,icd,deaths,Population,age_adjusted_rate,age_strat


In [15]:
dd.dtypes

State                        object
date                 datetime64[ns]
icd                          object
deaths                        int64
Population                   object
age_adjusted_rate           float64
age_strat                    object
dtype: object

In [5]:
dd = test.data['all']

In [ ]:
numeric_columns = []
def processor(
        x: pd.DataFrame,
        force_numeric: List[str] = ["population", "Crude Rate"],
    ) -> pd.DataFrame:
        """Process dataframes: convert columns dtype, compute new features.
        Args:
            x (pd.DataFrame): dataframe that we want to process
            force_numeric (List[str], optional): force these columns
                into numerical columns.
                Defaults to ["population", "Crude Rate"].

        Returns:
            pd.DataFrame: processed dataframe

        """

        # Automatically convert numeric columns to float/int type
        if numeric_columns.empty:
            numeric_columns = x.apply(
                lambda s: pd.to_numeric(s.replace(np.nan, 0), errors="coerce")
                .notnull()
                .all()
            )

            # compare with the force numeric and prevent numeric columns
            convert_cols = [
                col
                for col, bool_ in self.numeric_columns.items()
                if (bool_ or col in force_numeric)  # and col != "year"
                # keep year as str to prevent unexpected ticks on plots
            ]

        x.columns = numeric_columns.index

        x.date = x.date#.str.extract("(\d+)")
      
        x[convert_cols] = x[convert_cols].apply(
            pd.to_numeric, errors="coerce"
        )

        #x = x.loc[
        #    ~pd.concat(
        #        [x.eq(forbidden).any(axis=1) for forbidden in self.reject_list], axis=1
        #    ).any(axis=1)
        #]

['death 2010_2015 all.txt',
 'death 2010_2015 65+.txt',
 'death 2016_2017 all.txt',
 'death 2016_2017 65+.txt',
 'death 2018_2020 all.txt',
 'death 2018_2020 65+.txt']

In [ ]:
data_folder = "data/CDC"

available_files = os.listdir(data_folder)
raw_data = [file for file in available_files if "death" in file]
drop_cols = []

data = {
    (entry.split()[-1].split(".")[0], entry.split()[-2]): entry
    for entry in raw_data
}

age_strats = set(key[0] for key in data.keys()) 

dataframes = {
    age_strat: pd.concat(
        map(
            processor,
            [
                self.file_to_dataframe(self.data_folder, file).assign(
                    age_strat=age_strat
                )
                for key_tuple, file in data.items()
                if key_tuple[0] == age_strat
            ],
        ),
        axis=0,
    ).drop(columns=drop_cols)
    for age_strat in age_strats
}

with open(f"{data_folder}/{file}", "r") as f:
            for line in iter(lambda: f.readline().rstrip(), '"---"'):
                #CDC wonder add a total line despite we did not ask, 
                #ading one column sometime and breaking the pipeline
                if "Total" in line:
                    continue
                lines.append(process_line(line))
        # column is on the header, remove corner named notes
        res = pd.DataFrame(lines[1:], columns=lines[0][1:]).rename(
            columns=rename_mapper
        )
        # If Age Adjusted Rate is missing, fill with NaN
        if "age_adjusted_rate" not in res.columns:
            res["age_adjusted_rate"] = np.nan
        return res

NameError: name 'self' is not defined

In [ ]:
res = []
for file in raw_data:
  
    process_line = lambda line: line.strip().replace('"', "").split("\t")
    rename_mapper = {
                "Month Code": "date",
                "ICD Chapter": "icd",
                "Deaths": "deaths",
                "UCD - ICD Chapter": "UCD - ICD Chapter Code",
            }
    lines = []
    with open(f"{data_folder}/{file}", "r") as f:
        for line in iter(lambda: f.readline().rstrip(), '"---"'):
            if "Total" in line:
                continue
            lines.append(process_line(line))
    # column is on the header, remove corner named notes
    res.append(pd.DataFrame(lines[1:], columns=lines[0][1:]))#.rename(
       # columns=rename_mapper
    #)

 

In [ ]:
file = raw_data[2]
process_line = lambda line: line.strip().replace('"', "").split("\t")
rename_mapper = {
            "Month Code": "date",
            "ICD Chapter": "icd",
            "Deaths": "deaths",
            "UCD - ICD Chapter": "UCD - ICD Chapter Code",
        }
lines = []
with open(f"{data_folder}/{file}", "r") as f:
    i=0
    for line in iter(lambda: f.readline().rstrip(), '"---"'):
        lines.append(process_line(line))
        i+=1
        if i == 5:
            break
line

'\t"Alabama"\t"01"\t"Jan., 2016"\t"2016/01"\t"Endocrine, nutritional and metabolic diseases"\t"E00-E88"\t194\tNot Applicable\tNot Applicable'

In [ ]:
len(line.strip().replace('"', "").split("\t"))

9

In [ ]:
len(line.strip().replace('"', "").split("\t"))

9

In [ ]:
"Notes"	"State"	"State Code"	"Month"	"Month Code"	"UCD - ICD Chapter"	"UCD - ICD Chapter Code"	Deaths	Population	Crude Rate
"Notes"	"State"	"State Code"	"Month"	"Month Code"	"ICD Chapter"	"ICD Chapter Code"	Deaths	Population	Crude Rate